# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770902606321                   -0.52    9.0    173ms
  2   -2.772148440179       -2.90       -1.32    1.0    114ms
  3   -2.772170349925       -4.66       -2.47    1.0    109ms
  4   -2.772170653399       -6.52       -3.18    1.0    110ms
  5   -2.772170722133       -7.16       -3.98    2.0    130ms
  6   -2.772170722437       -9.52       -4.11    1.0    116ms
  7   -2.772170722993       -9.26       -4.87    1.0    114ms
  8   -2.772170723009      -10.78       -5.14    1.0    122ms
  9   -2.772170723015      -11.26       -6.22    1.0    121ms
 10   -2.772170723015      -12.92       -6.11    2.0    152ms
 11   -2.772170723015      -13.30       -6.50    1.0    124ms
 12   -2.772170723015      -13.72       -7.14    1.0    140ms
 13   -2.772170723015      -13.70       -7.56    1.0    136ms
 14   -2.772170723015   +  -13.67       -7.91    2.0    143ms
 15   -2.

1.7735579671161017

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770783549717                   -0.52    9.0    185ms
  2   -2.772060936365       -2.89       -1.32    1.0    107ms
  3   -2.772083065833       -4.66       -2.46    1.0    130ms
  4   -2.772083336281       -6.57       -3.14    1.0    111ms
  5   -2.772083416681       -7.09       -3.99    2.0    141ms
  6   -2.772083417678       -9.00       -4.58    1.0    114ms
  7   -2.772083417806       -9.90       -5.72    1.0    215ms
  8   -2.772083417810      -11.34       -5.79    2.0    184ms
  9   -2.772083417811      -12.38       -6.40    1.0    475ms
 10   -2.772083417811      -13.55       -7.62    2.0    157ms
 11   -2.772083417811      -13.85       -8.31    1.0    404ms

Polarizability via ForwardDiff:       1.7725349608243555
Polarizability via finite difference: 1.7735579671161017
